In [38]:
from dotenv import load_dotenv

load_dotenv()

True

In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
try:
    # Prefer Pydantic v2
    from pydantic import BaseModel, Field
except Exception:
    # Fallback for older stacks (not recommended for new code)
    from langchain_core.pydantic_v1 import BaseModel, Field

In [5]:
import os
import json
import pickle
from glob import glob

from typing import Dict, List, Any, Optional
from dataclasses import dataclass

from langchain_openai import OpenAIEmbeddings, AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Markdown, display

In [6]:
from prompt_templates.narratives import create_narrative_angles_prompt_template
from prompt_templates.Future_plan import create_future_plan_prompt_template
from prompt_templates.Activity import create_activity_list_generator_prompt_template
from prompt_templates.Main_essay import create_main_essay_ideas_prompt_template

In [41]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from tools import (
    generate_narrative_angles,
    generate_future_plan,
    generate_activity_list,
    generate_main_essay_ideas,
)

In [8]:
embeddings_model = OpenAIEmbeddings()

llm = AzureChatOpenAI(
    deployment_name="gpt-4o",
)

In [ ]:
# folder_path = "clab_data"
# applications_data = []

# # Get all JSON files in the folder
# for file_path in glob(os.path.join(folder_path, "*.json")):
#     with open(file_path, "r", encoding="utf-8") as f:
#         applications_data = json.load(f)
# print(f"Loaded {len(applications_data)} applications from {len(glob(os.path.join(folder_path, '*.json')))} files")

Loaded 8 applications from 13 files


In [9]:
with open("clab_data/all_data.json", "r", encoding="utf-8") as f:
    applications_data = json.load(f)

print(f"Loaded {len(applications_data)} applications from all_data.json")

Loaded 8 applications from all_data.json


In [10]:
@dataclass
class Application:  # This will store accepted applications as well as their embeddings
    filename: str
    content: Dict[str, Any]
    embedding: List[float] = None

In [11]:
applications = []

for key, application in applications_data.items():
    embedding = embeddings_model.embed_query(json.dumps(application))

    application = Application(
        filename=key,
        content=application,
        embedding=embedding,
    )

    applications.append(application)

with open("application_embeddings.pkl", "wb") as f:
    pickle.dump(applications, f)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [12]:
applications = pickle.load(open("application_embeddings.pkl", "rb"))

In [13]:
DUMMY_USER_DATA = {
    "name": "Sarah Johnson",
    "gender": "Female",
    "birth_country": "United States",
    "cultural_background": [
        "American",
        "first-generation college student",
        "rural upbringing",
        "STEM-focused",
        "environmental advocate",
    ],
    "academic": {
        "gpa": "3.95",
        "school_type": "Public High School",
        "test_scores": {"SAT": "1450", "ACT": "32"},
    },
    "activities": [
        {
            "position": "President",
            "category": "Academic",
            "description": "Led environmental science club, organized community clean-up events, coordinated with local government on sustainability initiatives",
        },
        {
            "position": "Team Captain",
            "category": "Athletics",
            "description": "Varsity soccer team captain, led team to state championship, organized team community service projects",
        },
        {
            "position": "Volunteer",
            "category": "Community Service",
            "description": "Tutored underprivileged students in math and science, developed curriculum for after-school program",
        },
        {
            "position": "Research Assistant",
            "category": "Research",
            "description": "Assisted professor in environmental impact study, analyzed data on renewable energy adoption in rural communities",
        },
    ],
    "future_plans": [
        "Study environmental engineering",
        "Work on renewable energy solutions",
        "Masters degree",
        "Start a non-profit focused on rural sustainability",
    ],
}

In [14]:
user_embedding = embeddings_model.embed_query(json.dumps(DUMMY_USER_DATA))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [15]:
similarities = []

for app in applications:
    similarity = cosine_similarity([user_embedding], [app.embedding])[0][0]
    similarities.append((app, similarity))

top_3_similar = sorted(similarities, key=lambda x: x[1], reverse=True)[:3]

print("Top 3 Accepted Applications:")
print("\n".join([each[0].filename for each in top_3_similar]))

Top 3 Accepted Applications:
Princeton_app1
Harvard_app1
Princeton_app2


In [16]:
application_narratives = []

for each in top_3_similar:
    temp = each[0].content

    if "application_narrative" in temp:
        application_narratives.append(temp["application_narrative"])

Narrative

In [84]:
narrative_angles_prompt_template = create_narrative_angles_prompt_template()

theme_chain = narrative_angles_prompt_template | llm | StrOutputParser()

theme_result = theme_chain.invoke(
    {
        "student_profile": json.dumps(DUMMY_USER_DATA)
        # "similar_narrative_themes": json.dumps(application_narratives),
    }
)

print(theme_result)
# wrap JSON parsing:
try:
    theme_result = json.loads(theme_result)
except json.JSONDecodeError as e:
    print(f"JSON parsing error: {e}")
    print(f"Raw output: {theme_result}")

2025-08-23 16:54:17,670 - httpx - INFO - HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


{
  "narrative_angles": [
    {
      "title": "From Soccer Field to Sustainability: Turning Teamwork into Renewable Energy Solutions",
      "positioning": "How does a soccer captain become a rural sustainability advocate?",
      "essay_concept": "Opening with a scene from the state championship → Connecting leadership on the field to her passion for community action → Realizing small rural innovations ripple globally → Developing skills through academic and research pursuits → Future vision of inclusive renewable energy systems.",
      "anchor_scene": "The roar of the crowd echoes, cleats pounding on the turf as Sarah leads her team to victory; afterward, she gathers her teammates for a community cleanup, beaming as they haul away mountains of waste together.",
      "unexpected_twist": "Combining sports leadership with scientific rigor to approach sustainability with a collaborative ethos.",
      "signature_initiative": {
        "name": "Rural Solar Soccer Fields",
        "desc

Future plan -


In [ ]:
future_plan_prompt_template = create_future_plan_prompt_template()

future_plan_chain = future_plan_prompt_template | llm | StrOutputParser()

future_plan_result = future_plan_chain.invoke(
    {
        "user_profile": json.dumps(DUMMY_USER_DATA),
        "narrative": theme_result["narrative_angles"][0],
    }
)

print(future_plan_result)

Environmental engineer, uniting rural athletes to pioneer renewable energy equity in underserved towns.


Activity-generator


In [ ]:
activity_prompt_template = create_activity_list_generator_prompt_template()
# Create the chain
activity_chain = activity_prompt_template | llm | StrOutputParser()

# Generate the activity list
activity_result = activity_chain.invoke(
    {
        "user_profile": json.dumps(DUMMY_USER_DATA),
        "narrative": theme_result["narrative_angles"][0],
        "future_plan": future_plan_result,
    }
)

print(activity_result)

**Enhanced Version of Existing:**

- **Current:** Led environmental science club, organized community clean-up events, coordinated with local government on sustainability initiatives  
- **Elevated:** Led regional sustainability campaigns, educated 400+ citizens, enacted waste-reduction policies with council, launched reusable packaging drive  
- **Position:** Regional Eco-Campaign Leader  
- **Organization:** CleanFuture Collective  
- **Description:** Led 20+ eco-events; educated 400+ citizens; reduced landfill waste by 15% via reusable packaging program in rural stores  

- **Current:** Varsity soccer team captain, led team to state championship, organized team community service projects  
- **Elevated:** State champion captain; implemented carbon-neutral championship protocol; led 80 athletes in sports sustainability workshops for 3 local schools  
- **Position:** Carbon-Neutral Sports Leader  
- **Organization:** GreenPlay Sports Initiative  
- **Description:** Implemented carbon-

main essay -


In [ ]:
main_essay_prompt_template = create_main_essay_ideas_prompt_template()

# Create the chain
main_essay_chain = main_essay_prompt_template | llm | StrOutputParser()

# Generate the main essay ideas
main_essay_result = main_essay_chain.invoke(
    {
        "user_profile": json.dumps(DUMMY_USER_DATA),
        "narrative": theme_result["narrative_angles"][0],
        "future_plan": future_plan_result,
        "activity_result": activity_result,
    }
)
print(main_essay_result)

```json
{
    "main_essay_ideas": [
        {
            "title": "The Carbon-Neutral Championship",
            "theme": "Leadership through sustainability and teamwork",
            "hook": "The aroma of burgers and fries filled the air as fans cheered, but I couldn’t focus on the trophy in my hands. My eyes were on the plastic cups and wrappers piling up around the field. A state championship was a victory—but at what cost to the environment?",
            "challenge": "As soccer captain, I realized our team’s triumphs were inadvertently leaving behind a trail of waste—both on the field and in our community.",
            "journey": "I convinced my teammates to implement carbon-neutral protocols for our games, from reusable water bottles to composting food waste. We held workshops for the younger teams, turning our field into a model of sustainability.",
            "growth": "I learned that leadership goes beyond the scoreboard; it’s about reshaping the culture around you, even wh

In [42]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an elite admissions strategist. "
     "When calling any tool, ALWAYS include the `user_profile` argument. "
     "Follow this strict sequence: "
     "1) generate_narrative_angles(user_profile) "
     "2) generate_future_plan(user_profile, narrative=<first angle>) "
     "3) generate_activity_list(user_profile, narrative=<first angle>, future_plan) "
     "4) generate_main_essay_ideas(user_profile, narrative=<first angle>, future_plan, activity_result). "
     "Return the final JSON from the last tool."),
    ("user", "USER_PROFILE(JSON): {user_profile}\n\nTASK: {input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

tools = [
    generate_narrative_angles,
    generate_future_plan,
    generate_activity_list,
    generate_main_essay_ideas,
]

agent = create_openai_tools_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [111]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = create_openai_tools_agent(llm, tools, prompt)

executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)


In [46]:
resp = executor.invoke({
    "input": "Generate full admissions workflow for this student",
    "user_profile": DUMMY_USER_DATA   
})
print(resp["output"])



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"



Invoking: `generate_narrative_angles` with `{'user_profile': "{'name': 'Sarah Johnson', 'gender': 'Female', 'birth_country': 'United States', 'cultural_background': ['American', 'first-generation college student', 'rural upbringing', 'STEM-focused', 'environmental advocate'], 'academic': {'gpa': '3.95', 'school_type': 'Public High School', 'test_scores': {'SAT': '1450', 'ACT': '32'}}, 'activities': [{'position': 'President', 'category': 'Academic', 'description': 'Led environmental science club, organized community clean-up events, coordinated with local government on sustainability initiatives'}, {'position': 'Team Captain', 'category': 'Athletics', 'description': 'Varsity soccer team captain, led team to state championship, organized team community service projects'}, {'position': 'Volunteer', 'category': 'Community Service', 'description': 'Tutored underprivileged students in math and science, developed curriculum for after-school program'}, {'position': 'Research Assistant', 'cate

INFO:httpx:HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


{'error': 'Invalid JSON from model', 'raw': '```json\n{\n  "narrative_angles": [\n    {\n      "title": "From Rural Soccer Fields to Renewable Energy Frontlines",\n      "positioning": "A STEM-driven environmental advocate channels the grit of a soccer captain into revolutionizing rural sustainability.",\n      "essay_concept": "Opening scene: A muddy soccer field surrounded by wind turbines → Personal connection: Balancing soccer captainship with leading clean-up events → Broader realization: Rural communities need sustainable solutions tailored to their challenges → Action: Conducting renewable energy adoption research for rural areas → Future vision: Advancing environmental engineering to bring self-sustaining energy to underrepresented regions.",\n      "anchor_scene": "Under a gray sky, Sarah wipes dirt from her soccer cleats as the distant hum of a wind turbine fills the air. Her team huddles around her, preparing a recycled-themed fundraiser after practice. It hits her: rural po

...

INFO:httpx:HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


1. **Enhanced Version of Existing:**
   - Current: Led environmental science club, organized community clean-up events, coordinated with local government on sustainability initiatives  
   - Elevated: Launched youth-led "Eco-Rural Initiative"; hosted 10 clean-ups; advised county on waste-to-compost program; mentored 15+ future green leaders  
   - Position: Founder, Eco-Rural Initiative  
   - Organization: Green Futures Club  
   - Description: Founded initiative advising towns on waste-to-compost plans; scaled program to 3 counties & trained 15+ student sustainability advocates.  

   - Current: Varsity soccer team captain, led team to state championship, organized team community service projects  
   - Elevated: Merged soccer & service; won state title; ran "Cleats for Cleanups," sending 200+ recycled cleats abroad; shifted team into sustainability ambassadors  
   - Position: Captain & Sustainability Lead  
   - Organization: Rural High Varsity Soccer  
   - Description: Linked ath

INFO:httpx:HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"



Invoking: `generate_main_essay_ideas` with `{'user_profile': "{'name': 'Sarah Johnson', 'gender': 'Female', 'birth_country': 'United States', 'cultural_background': ['American', 'first-generation college student', 'rural upbringing', 'STEM-focused', 'environmental advocate'], 'academic': {'gpa': '3.95', 'school_type': 'Public High School', 'test_scores': {'SAT': '1450', 'ACT': '32'}}, 'activities': [{'position': 'President', 'category': 'Academic', 'description': 'Led environmental science club, organized community clean-up events, coordinated with local government on sustainability initiatives'}, {'position': 'Team Captain', 'category': 'Athletics', 'description': 'Varsity soccer team captain, led team to state championship, organized team community service projects'}, {'position': 'Volunteer', 'category': 'Community Service', 'description': 'Tutored underprivileged students in math and science, developed curriculum for after-school program'}, {'position': 'Research Assistant', 'cate

ValidationError: 1 validation error for MainEssayIdeasInput
narrative
  Input should be a valid dictionary [type=dict_type, input_value='{"title": "From Rural So...e Energy Engineering"]}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type

In [24]:
resp = executor.invoke({
    "input": "Generate full admissions workflow for this student",
    "user_profile": DUMMY_USER_DATA   
})
print(resp["output"])



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"



Invoking: `generate_narrative_angles` with `{'user_profile': "{'name': 'Sarah Johnson', 'gender': 'Female', 'birth_country': 'United States', 'cultural_background': ['American', 'first-generation college student', 'rural upbringing', 'STEM-focused', 'environmental advocate'], 'academic': {'gpa': '3.95', 'school_type': 'Public High School', 'test_scores': {'SAT': '1450', 'ACT': '32'}}, 'activities': [{'position': 'President', 'category': 'Academic', 'description': 'Led environmental science club, organized community clean-up events, coordinated with local government on sustainability initiatives'}, {'position': 'Team Captain', 'category': 'Athletics', 'description': 'Varsity soccer team captain, led team to state championship, organized team community service projects'}, {'position': 'Volunteer', 'category': 'Community Service', 'description': 'Tutored underprivileged students in math and science, developed curriculum for after-school program'}, {'position': 'Research Assistant', 'cate

INFO:httpx:HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


{'narrative_angles': [{'title': "From Soccer Fields to Solar Fields: A Rural Advocate's STEM Journey", 'positioning': 'How does a varsity athlete become a future pioneer of rural renewable energy solutions?', 'essay_concept': 'Opening on the sweeping beauty of her rural hometown → Connection to her childhood admiration for nature and community soccer life → Realization that environmental challenges threaten rural vitality → Action through environmental advocacy and clean-up projects → Vision to create sustainable solutions combining STEM and community values.', 'anchor_scene': 'The warmth of sunset painting the soccer field as Sarah rallies her team for a community clean-up project; laughter merges with the sound of garbage bags rustling, and the crystal-clear air reminds her why protecting rural spaces feels urgent and personal.', 'unexpected_twist': 'The juxtaposition of a soccer captain leading sustainable initiatives positions Sarah’s athletic team as a surprising vehicle for envir

...

INFO:httpx:HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


**Enhanced Version of Existing:**

1. **Current:** Led environmental science club, organized community clean-up events, coordinated with local government on sustainability initiatives  
   **Elevated:** Led STEM workshops linking rural students to local sustainability efforts, co-designed solar-powered irrigation pilot with officials  
   **Position:** Environmental Innovator & Club President  
   **Organization:** Green Revival Youth Initiative  
   **Description:** Built coalition w/ officials; launched solar irrigation in rural farms; empowered 85 students w/ STEM project workshops.  

2. **Current:** Varsity soccer team captain, led team to state championship, organized team community service projects  
   **Elevated:** Pioneered “Eco Sports” concept blending athletics/community cleanup; team influential in banning single-use plastics at stadium  
   **Position:** Eco-Athlete & Team Leader  
   **Organization:** Clean Athletic Outreach Alliance  
   **Description:** Led team to sta

INFO:httpx:HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"



Invoking: `generate_main_essay_ideas` with `{'user_profile': "{'name': 'Sarah Johnson', 'gender': 'Female', 'birth_country': 'United States', 'cultural_background': ['American', 'first-generation college student', 'rural upbringing', 'STEM-focused', 'environmental advocate'], 'academic': {'gpa': '3.95', 'school_type': 'Public High School', 'test_scores': {'SAT': '1450', 'ACT': '32'}}, 'activities': [{'position': 'President', 'category': 'Academic', 'description': 'Led environmental science club, organized community clean-up events, coordinated with local government on sustainability initiatives'}, {'position': 'Team Captain', 'category': 'Athletics', 'description': 'Varsity soccer team captain, led team to state championship, organized team community service projects'}, {'position': 'Volunteer', 'category': 'Community Service', 'description': 'Tutored underprivileged students in math and science, developed curriculum for after-school program'}, {'position': 'Research Assistant', 'cate

ValidationError: 1 validation error for MainEssayIdeasInput
narrative
  Input should be a valid dictionary [type=dict_type, input_value='{"title": "From Soccer F...nd Regional Planning"]}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type

In [ ]:
from workflow_orchestrator import generate_application_strategy

# Use the orchestrator instead of individual chains
result = generate_application_strategy(DUMMY_USER_DATA)

# Or use the agent with the new tools
from tools import generate_complete_application_strategy

# Add this tool to your agent
tools = [generate_complete_application_strategy]

2025-08-23 16:00:17,032 - workflow_orchestrator - INFO - WorkflowOrchestrator initialized successfully
2025-08-23 16:00:17,033 - workflow_orchestrator - INFO - Starting complete workflow execution
2025-08-23 16:00:17,035 - workflow_orchestrator - INFO - Starting narrative angles generation
2025-08-23 16:00:37,800 - httpx - INFO - HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-08-23 16:00:37,806 - workflow_orchestrator - INFO - Function _generate_narratives executed successfully in 20.77s
2025-08-23 16:00:37,808 - workflow_orchestrator - INFO - Narrative angles generation completed with status: WorkflowStatus.VALIDATED
2025-08-23 16:00:37,810 - workflow_orchestrator - INFO - Starting future plan generation
2025-08-23 16:00:38,480 - httpx - INFO - HTTP Request: POST https://clabai3366454354.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 

In [63]:
import logging

from workflow_orchestrator import WorkflowOrchestrator, generate_application_strategy

# Configure logging to see the workflow progress
logging.basicConfig(level=logging.INFO)